In [8]:
from aurora_postgres_helpers import SQLConnection
from transformation_helpers import *

/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:197: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if hr is 0:


In [4]:
sql = SQLConnection
staging_schema = 'yusra_stories_staging'
production_schema = 'yusra_stories_production'
logs_table = 'logs'

In [7]:
logs_df = sql.read_table_to_df(sql, staging_schema, logs_table)

SQL READ from yusra_stories_staging.logs


,ride_id,log
0,1,--------- beginning of a new ride\n
1,1,2022-10-07 13:53:01.268572 mendoza v9: Getting...
2,1,2022-10-07 13:53:01.769243 mendoza v9: [SYSTEM...
3,1,2022-10-07 13:53:02.269911 mendoza v9: [INFO]:...
4,1,2022-10-07 13:53:02.770593 mendoza v9: [INFO]:...
...,...,...
1038,1,2022-10-07 14:01:40.474608 mendoza v9: [INFO]:...
1039,1,2022-10-07 14:01:40.975284 mendoza v9: [INFO]:...
1040,1,2022-10-07 14:01:41.475977 mendoza v9: [INFO]:...
1041,1,2022-10-07 14:01:41.976642 mendoza v9: [INFO]:...


In [ ]:
formatted_df = get_joined_formatted_df(logs_df)